In [33]:
# 모듈 로딩

import pandas as pd

In [34]:
# 데이터 준비

data1 = pd.read_excel("2조 괴담 파일.xlsx", sheet_name='BJY', header= 1)
data2 = pd.read_excel("2조 괴담 파일.xlsx", sheet_name='LHG_Neapolitan', header= 1)
data3 = pd.read_excel("2조 괴담 파일.xlsx", sheet_name= 'LHG_Horror', header= 1)
data4 = pd.read_excel("2조 괴담 파일.xlsx", sheet_name= 'SY', header = 1)

In [35]:
# 괴담 데이터 하나의 데이터프레임으로 합치기

horror = pd.concat([data1, data2, data3, data4], axis = 0)
horror.head()

,CLASS,TYPE,FROM,TEXT
0,H,E,K,"""그런데 어떻게 알았어?"" 이렇게 묻더라구요, 저는 뭐가? 라고 대답하니 ""꿈인줄 ..."
1,H,E,K,또 똑같은 꿈을 꿨습니다. 방에서 일어나니까 옆방에서 여자가 미친듯이 웃으면서 00...
2,H,E,K,그리고 출근 한 다음 근무 후 새벽에 퇴근하고 아침에 잠들었는데 낮 열두시쯤 까지 ...
3,H,E,K,여튼 때는 20대 중반이었나?? 여름이었던것 같은데 여느때처럼 내방에서 잠을 자고 ...
4,H,E,K,온몸에 소름이 돋으면서 식은땀 ㅈㄴ나고 뭐지?? 도둑인가?? 이런 생각에 너무 무서...


In [36]:
#나는 type만 사용할 거니까 나머지는 제거
horror = horror.drop(['CLASS', 'FROM'], axis = 1)

horror.head() #없어져땅. 굿~

,TYPE,TEXT
0,E,"""그런데 어떻게 알았어?"" 이렇게 묻더라구요, 저는 뭐가? 라고 대답하니 ""꿈인줄 ..."
1,E,또 똑같은 꿈을 꿨습니다. 방에서 일어나니까 옆방에서 여자가 미친듯이 웃으면서 00...
2,E,그리고 출근 한 다음 근무 후 새벽에 퇴근하고 아침에 잠들었는데 낮 열두시쯤 까지 ...
3,E,여튼 때는 20대 중반이었나?? 여름이었던것 같은데 여느때처럼 내방에서 잠을 자고 ...
4,E,온몸에 소름이 돋으면서 식은땀 ㅈㄴ나고 뭐지?? 도둑인가?? 이런 생각에 너무 무서...


### 데이터 전처리하기

In [37]:
import string
punc = string.punctuation

# 정규표현식 패턴만들기
hangle_pattern = "[^ㄱ-ㅎㅏ-ㅣ가-힣]"
num_pattern = "[0-9]"
eng_pattern = "[a-zA-Z]"

# 한글, 숫자, 영어만 남기기
horror['TEXT'] = horror['TEXT'].str.replace(hangle_pattern, '', regex=True)
horror['TEXT'] = horror['TEXT'].str.replace(num_pattern, '', regex=True)
horror['TEXT'] = horror['TEXT'].str.replace(eng_pattern, '', regex=True)

In [38]:
# 빈 문장 있으면 제거
import numpy as np
horror['TEXT'].replace('', np.nan, inplace=True)
horror = horror.dropna()

In [39]:
#불용어 사전 불러오기
from urllib.request import urlretrieve

filename = '../datas/kr_stopwords.txt'
url = "https://gist.githubusercontent.com/chulgil/d10b18575a73778da4bc83853385465c/raw/a1a451421097fa9a93179cb1f1f0dc392f1f9da9/stopwords.txt"
ret = urlretrieve(url, filename)

with open(filename, 'r', encoding='utf-8') as f:
    stopwords = set(f.read().splitlines())

In [40]:
#토크나이저
from konlpy.tag import Okt
okt = Okt()

In [41]:
# 데이터 분할하기(train, test)
from sklearn.model_selection import train_test_split
train, test = train_test_split(horror, test_size=0.2, random_state=42)

In [42]:
test.isin(stopwords).sum()

TYPE    0
TEXT    0
dtype: int64

In [43]:
train.shape, test.shape

((2560, 2), (640, 2))

In [44]:
for sentence in train['TEXT']:
    print(type(sentence))
    print(okt.morphs(sentence))
    break

<class 'str'>
['관리', '실', '에서', '여성', '직원', '이', '당신', '의', '휴대폰', '으로', '어디', '로', '와', '달라', '는', '등', '의', '요청', '이', '왔다면', '바로', '끊으십시오', '관리', '실', '에선', '어떤', '경우', '에서도', '당신', '의', '휴대폰', '에', '직접', '전화', '를', '하지', '않을것이며', '관리', '실', '에서는', '여성', '직원', '을', '고용', '하', '지', '않습니다']


In [45]:
# 토크나이즈 이후 불용어 제거하기
train_tokens = [okt.morphs(sentence)for sentence in train['TEXT']] # tokenize
train_tokens = [[token for token in sent if token not in stopwords]for sent in train_tokens] # extract stopwords
test_tokens = [okt.morphs(sentence)for sentence in test['TEXT']] #tokenize
test_tokens = [[token for token in sent if token not in stopwords]for sent in test_tokens] #extract stopwords

In [46]:
backup_train = train_tokens
backup_test = test_tokens

In [47]:
test_tokens[0]

['할아버지',
 '돌아가신',
 '후난',
 '추억',
 '떠올리며',
 '유품',
 '정리',
 '하고',
 '있었',
 '다그',
 '오래된',
 '포르노',
 '컬렉션',
 '살펴보던난',
 '내게',
 '여동생',
 '있었다는',
 '사실',
 '깨닫게',
 '되었다']

In [48]:
# train_tokens = backup_train
# test_tokens = backup_text

In [49]:
# 단어사전 생성하기
vocab = {'<PAD>': 8000, '<UNK>':7999}
for sen in train_tokens:
    for word in sen:
        if word in vocab:
            vocab[word] += 1
        else:
            vocab[word] = 1
        
# 집합으로 변환하여 패딩 값 추가 

# vocab2 = {}
# for sen in test_tokens:
#     for word in sen:
#         if word is not None:  # None이 아닌 경우에만 처리
#             if word in vocab:
#                 vocab2[word] += 1
#             else:
#                 vocab2[word] = 1

# 결과 확인
# 패드가 없어서 계속 none 값이 섞이길래 강제로 추가했다.


In [50]:
sorted(vocab.items(), key=lambda x: x[1], reverse=True)[:10]

[('<PAD>', 8000),
 ('<UNK>', 7999),
 ('는', 1919),
 ('은', 1629),
 ('한', 1121),
 ('도', 714),
 ('내', 541),
 ('다', 513),
 ('말', 490),
 ('하고', 456)]

In [51]:
# 불용어 처리 한번 더,,,,,,,,,,
clean = {token: value for token,value in vocab.items() if token not in stopwords}

sorted(clean.items(), key=lambda x: x[1], reverse=True)[:10]

[('<PAD>', 8000),
 ('<UNK>', 7999),
 ('는', 1919),
 ('은', 1629),
 ('한', 1121),
 ('도', 714),
 ('내', 541),
 ('다', 513),
 ('말', 490),
 ('하고', 456)]

In [52]:
# 인코딩 / 디코딩 딕셔너리 생성하기

encode = {token: idx for idx, token in enumerate(vocab)}
decode = {idx: token for idx, token in enumerate(vocab)}

UNK = encode.get('<UNK>')

In [53]:
# 문장 인코딩하기

train_id = [[encode.get(token, UNK) for token in sen]for sen in train_tokens]
test_id = [[encode.get(token,UNK) for token in sen] for sen in test_tokens]


In [54]:
len(train_id), len(test_id)

(2560, 640)

In [55]:
test_id[0]

[720,
 722,
 1,
 9105,
 1,
 2132,
 1840,
 435,
 3851,
 283,
 6905,
 1,
 4173,
 1,
 858,
 2057,
 5503,
 941,
 3824,
 244]

In [56]:
#PAD

def pad_sequences(sequences, max_len, pad_token):
    padded = []
    for seq in sequences:
        if seq is None:  # 시퀀스가 None일 경우 패딩하지 않고 건너뜁니다.
            continue
        if len(seq) < max_len:
            seq = seq + [pad_token] * (max_len - len(seq))
        else:
            seq = seq[:max_len]
        padded.append(seq)
    return padded


In [57]:
MAX_LEN = max([len(sen) for sen in train_id + test_id])
PAD_ID = encode.get('<PAD>')

train_id = pad_sequences(train_id, MAX_LEN, PAD_ID)
test_id = pad_sequences(test_id, MAX_LEN, PAD_ID)

print(f'TRAIN_ID: {train_id[0]},\n\n TEST_ID: {test_id[0]}')

TRAIN_ID: [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 2, 3, 12, 13, 14, 6, 15, 16, 17, 18, 2, 3, 19, 4, 5, 20, 21, 22, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],

 TEST_ID: [720, 722, 1, 9105, 1, 2132, 1840, 435, 3851, 283, 6905, 1, 4173, 1, 858, 2057, 5503, 941, 3824, 244

RNN 모델 구축하기

In [58]:
import torch
import torch.nn as nn

import numpy as np

In [59]:
# 각 train, test의 label과 text 나누기
xtrain = np.array(train_id)
ytrain = np.array(train['TYPE'])
xtest = np.array(test_id)
ytest = np.array(test['TYPE'])

In [60]:
xtest

array([[  720,   722,     1, ...,     0,     0,     0],
       [  867, 20207,   344, ...,     0,     0,     0],
       [  510,  7710,  8296, ...,     0,     0,     0],
       ...,
       [    1,  1248,  1818, ...,     0,     0,     0],
       [ 5707,     1, 15602, ...,     0,     0,     0],
       [ 6533,  1513,     9, ...,     0,     0,     0]])

In [61]:
ytrain

array(['G', 'G', 'E', ..., 'G', 'G', 'E'], dtype=object)

In [62]:
# 라벨이 정수가 아니라서 계속 오류가 생긴다. 제기랄.
# 라벨인코더도 안 먹어서 내가 그냥 바꿔주도록 하자.
label_to_index = {'G': 0, 'E': 1}

# ytrain 인코딩
y_train = [label_to_index[label] for label in train['TYPE']]
y_train = torch.FloatTensor(y_train)

# ytest 인코딩
y_test = [label_to_index[label] for label in test['TYPE']]
y_test = torch.FloatTensor(y_test)

xtrain = torch.LongTensor(xtrain)
xtest = torch.LongTensor(xtest)

In [63]:
# 단어사전 크기 계산하기
vocab_num = len(encode) + 1 #패딩 크기(1) 계산하기

vocab_num

23943

RNN 모델 정의하기

In [64]:
# # RNN

# class RNNClassifier(nn.Module):
#     def __init__(self, n_vocab, output_size, embedding_dim, hidden_dim, n_layers, dropout):
#         super().__init__()
#         self.embedding = nn.Embedding(n_vocab, embedding_dim)
#         self.rnn = nn.RNN(embedding_dim, hidden_dim, n_layers, batch_first=True, dropout=dropout)
#         self.fc = nn.Linear(hidden_dim, output_size)
        
#     def forward(self, x):
#         embedded = self.embedding(x)
#         output, hidden = self.rnn(embedded)
#         output = self.fc(output[:,-1,:])
#         return output
        
# model = RNNClassifier(vocab_num, output_size=2, embedding_dim=64, hidden_dim=128, n_layers=2, dropout=0.5)

In [65]:
# 모델 정의
class RNNClassifier(nn.Module):
    def __init__(self, n_vocab, embedding_dim, hidden_dim, n_layers, dropout):
        super().__init__()
        self.embedding = nn.Embedding(n_vocab, embedding_dim)
        self.rnn = nn.RNN(embedding_dim, hidden_dim, n_layers, batch_first=True, dropout=dropout)
        self.fc = nn.Linear(hidden_dim, 1)  # 출력 크기를 1로 변경 (이진 분류 문제)
        self.sigmoid = nn.Sigmoid()  # 시그모이드 함수

    def forward(self, x):
        embedded = self.embedding(x)
        output, hidden = self.rnn(embedded)
        output = self.fc(output[:, -1, :])
        output = self.sigmoid(output)  # 시그모이드 함수 적용
        return output

# 모델 생성
model = RNNClassifier(vocab_num, embedding_dim=64, hidden_dim=128, n_layers=2, dropout=0.5)

# 손실함수 및 옵티마이저 정의
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


In [66]:
# 손실함수, 옵티마이저 정의하기

criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1)

In [67]:
xtrain.dtype
y_train

tensor([0., 0., 1.,  ..., 0., 0., 1.])

In [68]:
from torch.utils.data import TensorDataset, DataLoader

# 데이터셋 및 데이터 로더 생성
train_dataset = TensorDataset(xtrain, y_train)
# ytest와 동일한 개수의 샘플을 xtest에서 선택하여 새로운 xtest_subset을 만듭니다.
xtest_subset = xtest[:len(y_test)]
test_dataset = TensorDataset(xtest_subset, y_test)
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)



In [69]:
len(xtest)

640

In [70]:
xtrain.shape, ytrain.shape, xtest_subset.shape, ytest.shape

(torch.Size([2560, 288]), (2560,), torch.Size([640, 288]), (640,))

In [71]:
xtest[0], ytest[0]

(tensor([ 720,  722,    1, 9105,    1, 2132, 1840,  435, 3851,  283, 6905,    1,
         4173,    1,  858, 2057, 5503,  941, 3824,  244,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,  

In [72]:
for f, t in train_dataset:
    print(f.shape, t.shape)
    break

torch.Size([288]) torch.Size([])


In [73]:
import torch.nn.functional as F
epochs = 10
for epoch in range(epochs):
    optimizer.zero_grad()
    outputs = model(xtrain)
    outputs = outputs.squeeze()
    loss = criterion(outputs, y_train)
    loss.backward()
    optimizer.step()
    
    # 평가
    outputs = model(xtest)
    
    # y_test를 one-hot 벡터로 변환
    # 모델 출력과 정수로 된 레이블 비교
    predicted_labels = outputs.argmax(dim=1)
    accuracy = (predicted_labels == y_test).float().mean()

    
    # 모델 출력과 one-hot 레이블 비교
    #accuracy = (outputs.argmax(dim=1) == y_one_hot.argmax(dim=1)).float().mean()
    
    print(f'Epoch: {epoch+1}, Loss: {loss.item()}, Accuracy: {accuracy.item()*100}%')

Epoch: 1, Loss: 0.7006478309631348, Accuracy: 53.125%
Epoch: 2, Loss: 3.1193807125091553, Accuracy: 53.125%
Epoch: 3, Loss: 22.29237937927246, Accuracy: 53.125%
Epoch: 4, Loss: 27.685970306396484, Accuracy: 53.125%
Epoch: 5, Loss: 32.16034698486328, Accuracy: 53.125%
Epoch: 6, Loss: 37.013893127441406, Accuracy: 53.125%
Epoch: 7, Loss: 40.64053726196289, Accuracy: 53.125%
Epoch: 8, Loss: 41.362422943115234, Accuracy: 53.125%
Epoch: 9, Loss: 42.21215057373047, Accuracy: 53.125%
Epoch: 10, Loss: 42.31328201293945, Accuracy: 53.125%


In [74]:
new_data = ['이 수칙서에 5번 문항은 존재하지 않습니다. 5번 문항이 존재하는 수칙서를 배급받았을 시 즉시 찢어버리고, 물가에는 3일간 가지 마세요.']
new_token = [okt.morphs(sentence) for sentence in new_data]
new_ids = [[encode.get(token, UNK) for token in sen]for sen in new_token]
new_ids = pad_sequences(new_ids, MAX_LEN, PAD_ID)
new_ids = torch.LongTensor(new_ids)

In [76]:
# 예측
model.eval()
with torch.no_grad():
    outputs = model(new_ids)
    _, predicted_labels = torch.max(outputs, dim=1)
    torch.save(model.state_dict(), f'../savepoint/model_scarystory.pth')
    

In [ ]:
for i, pred in enumerate(predicted_labels):
    print(f'입력된 문장 : {new_data[i]}')
    if pred== 0:
        print("추리형 괴담입니다.")
    else:
        print("경험담형 괴담입니다.")

입력된 문장 : 이 수칙서에 5번 문항은 존재하지 않습니다. 5번 문항이 존재하는 수칙서를 배급받았을 시 즉시 찢어버리고, 물가에는 3일간 가지 마세요.
추리형 괴담입니다.
